### Імпорт бібліотек

In [1]:
import pandas as pd

import requests # для парсингу сайта
from bs4 import BeautifulSoup

from geopy.distance import geodesic # для знаходження відстані між координатними точками

### Оцінка по районах

In [2]:
df_region = pd.read_csv('region_score.csv')
df_region.head()

,Назва,Total
0,Голосіїв,3
1,Нова Дарниця,1
2,Соцмісто,2
3,Оболонь,3
4,Печерськ,4


###  Відстань до метро

In [3]:
# Координати станцій метро
df_metro = pd.read_csv('metro_st.csv')
df_metro.head()

,Name,Latitude,Longitude
0,Академмістечко,50.465414,30.355455
1,Житомирська,50.456436,30.365407
2,Святошин,50.457804,30.390660
3,Нивки,50.459556,30.404600
4,Берестейська,50.460383,30.418808


In [4]:
def find_metro(df_metro, lat, lon):
    """Функція, яка обчислює відстань від квартири до найближчої станції метро
    return - відстань в км до найближчої станції метро"""
    
    metro = [(df_metro['Latitude'][i], df_metro['Longitude'][i]) for i in range(df_metro.shape[0])]
    my_address = (lat, lon)
    dist = list(map(lambda i: round(geodesic(i, my_address).km, 2), metro))
    return min(dist)

### Парсинг сайту 100realty.ua

In [5]:
HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }

In [27]:
region_list = []
region_num_list = []
area_list = []
rooms_list = []
ac_list = []
adress_list = []
price_list = []
coordinates = []
metro_list = []
metro_score_list = []

for page in range (0, 36):
    

    print(f'Parsing {page}')
        
#     SITE = f'https://100realty.ua/uk/realty_search/apartment/rent/nr_1,2,3,4,5_/cur_3/kch_2?page={page}'
    SITE = f'https://100realty.ua/uk/realty_search/apartment/rent/r_1000008,1000015,1003012,1003091/nr_1,2,3,4/p_5000_40000/cur_3/kch_2?page={page}'
    r = requests.get(SITE, headers = HEADERS)

    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')
        
        for el in soup.find_all('div', class_ = "object-common-wrapper"): 
            
            # Адреса
            adr = el.find('div', class_ = "object-address").text.strip()
            adress_list.append(adr)

            # Район
            # Категорія району
            try:
                region = el.find('div', class_ = "object-region").text.strip().split()[-2]
                region_list.append(region)
                region_score = df_region[df_region['Назва'] == region]['Total']              
                region_score = int(region_score)
                region_num_list.append(region_score)
        
            except IndexError as e:
                region_num_list.append(None)
            except TypeError:
                region_num_list.append(None)
              
            # Площа
            object_square = float(el.find('div', class_ = "object-square object-info-item").text.split()[-2])
            area_list.append(object_square)
            
            # Кількість кімнат
            rooms_number = int(el.find('div', class_ = "object-rooms object-info-item").text.split()[2])
            rooms_list.append(rooms_number)

            # ЖК
            # 1 - ЖК
            # 0 - не ЖК
            ac = el.find('div', class_ = "object-resident object-info-item") 
            if ac is not None:
                apartment_complex = ac.find('div', class_ = "value").text
                ac_list.append(1)
            else:
                ac_list.append(0)
                           
            # Ціна
            price = el.find('div', class_ = "cost-field").find('span').text.replace('грн./місяць','').replace('*','')
            if price.find(' '):
                price = price.replace(' ','')
            price_list.append(int(price))
            
            # Координати 
            link2='https://100realty.ua'+(el.find('div', class_='object-address').find('a').get('href'))
            r2 = requests.get(link2, headers=HEADERS)
            if r2.status_code == 200:
                soup2 = BeautifulSoup(r2.text, 'lxml')
                try:
                    lat = soup2.find('div', class_="object-map__content")['data-lat']
                    long = soup2.find('div', class_="object-map__content")['data-lng']
                    coordinates.append((lat, long))
                    
                    # Відстань до метро в км
                    metro_dist = find_metro(df_metro, lat, long)
                    metro_list.append(metro_dist)
                    
                    # Близкість до метро
                    # 1 - до 15 хв (до 1 км) 
                    # 2 - до 30 хв (до 2 км) 
                    # 3 - більше 30 хв (більше 2 км) 
                    
                    if metro_dist < 1:
                        metro_score_list.append(1)
                    elif metro_dist < 2:
                        metro_score_list.append(2)
                    else:
                        metro_score_list.append(3)
                        
                except TypeError as e:
                    coordinates.append(None)
                    metro_list.append(None)
                    metro_score_list.append(None)
                    
                except :
                    coordinates.append(None)
                    metro_list.append(None)
                    metro_score_list.append(None)

Parsing 0
Parsing 1
Parsing 2
Parsing 3
Parsing 4
Parsing 5
Parsing 6
Parsing 7
Parsing 8
Parsing 9
Parsing 10
Parsing 11
Parsing 12
Parsing 13
Parsing 14
Parsing 15
Parsing 16
Parsing 17
Parsing 18
Parsing 19
Parsing 20
Parsing 21
Parsing 22
Parsing 23
Parsing 24
Parsing 25
Parsing 26
Parsing 27
Parsing 28
Parsing 29
Parsing 30
Parsing 31
Parsing 32
Parsing 33
Parsing 34
Parsing 35


### Підготовка даних

In [32]:
# Створення датафрейму на основі даних з сайту

df = pd.DataFrame({
    'region': region_list, 
    'region score': region_num_list,
    'area': area_list, 
    'rooms': rooms_list, 
    'appartment complex': ac_list, 
    'price': price_list, 
    'adress': adress_list, 
    'coordinates': coordinates,
    'metro distance': metro_list,
    'metro score': metro_score_list})

In [33]:
df.head()

,region score,area,rooms,appartment complex,price,adress,coordinates,metro distance,metro score
0,4,18.0,1,0,11700,"вул. Хрещатик, 7/11, Київ","(50.4511322, 30.5264102)",0.28,1
1,4,75.0,3,0,39600,"вул. Хрещатик, Київ","(50.447186, 30.523071)",0.10,1
2,4,45.0,1,1,35700,"вул. Маккейна Джона (Кудрі Івана), 1, Київ","(50.4139129, 30.5264399)",0.16,1
3,4,94.0,3,0,39600,"вул. Тютюнника Василя (Барбюса Анрі), 37, Київ","(50.4211248, 30.5263316)",0.40,1
4,4,140.0,3,0,39600,"вул. Дегтярна, Київ","(50.4581896, 30.5104905)",0.64,1


In [31]:
# Збереження датасету в файл df_data.csv
df.to_csv('df_score4.csv', index=False)